# Libraries and SAM initialization

In [3]:

import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
# !{sys.executable} -m pip install opencv-python matplotlib
# !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'


# !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
# !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

PyTorch version: 2.2.2+cu121
Torchvision version: 0.17.2+cu121
CUDA is available: True


In [4]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import os
import shutil
from sklearn.cluster import KMeans
import copy
import joblib

In [6]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# sam_checkpoint = "sam_vit_h_4b8939.pth"
sam_checkpoint = "sam_vit_b_01ec64.pth"
model_type = "vit_b"


device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

# Functions

In [7]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [8]:
def returnMasks(image):
    mask_generator_2 = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=160,
    pred_iou_thresh=0.85,
    stability_score_thresh=0.85,
    # crop_n_layers=2,
    # crop_n_points_downscale_factor=3,
    min_mask_region_area=100,
    stability_score_offset = 1.0,
    box_nms_thresh = 0.7,
    # crop_nms_thresh = 0.7,
    )
    masks = mask_generator_2.generate(image)
    return masks

In [9]:
def findcolors(image):
    # Define color ranges for pink, white, purple, and black
    pink_lower = np.array([150, 50, 150], dtype=np.uint8)
    pink_upper = np.array([255, 150, 255], dtype=np.uint8)

    white_lower = np.array([200, 200, 200], dtype=np.uint8)
    white_upper = np.array([255, 255, 255], dtype=np.uint8)

    purple_lower = np.array([100, 0, 100], dtype=np.uint8)
    purple_upper = np.array([180, 100, 180], dtype=np.uint8)

    black_lower = np.array([0, 0, 0], dtype=np.uint8)
    black_upper = np.array([30, 30, 30], dtype=np.uint8)

    # Create masks for each color range
    pink_mask = cv2.inRange(image, pink_lower, pink_upper)
    white_mask = cv2.inRange(image, white_lower, white_upper)
    purple_mask = cv2.inRange(image, purple_lower, purple_upper)
    black_mask = cv2.inRange(image, black_lower, black_upper)

    # Calculate the number of pixels for each color
    np_pixels = np.sum(pink_mask > 0)
    nw_pixels = np.sum(white_mask > 0)
    npr_pixels = np.sum(purple_mask > 0)

    # Calculate percentages
    total_pixels = np_pixels+nw_pixels+npr_pixels  # Total number of pixels (assuming 3 channels)
    np_percentage = (np_pixels / total_pixels) * 100
    nw_percentage = (nw_pixels / total_pixels) * 100
    npr_percentage = (npr_pixels / total_pixels) * 100

    # Format the result as a string
    result_string = f"Pink: {np_percentage:.2f}%, White: {nw_percentage:.2f}%, Purple: {npr_percentage:.2f}%"

    return [np_percentage,nw_percentage,npr_percentage],result_string

In [10]:
def pltshow(image,title='',annotations=False,size=5):
    plt.figure(figsize=(size,size))
    plt.imshow(image)
    if(annotations):
        show_anns(masks)
    plt.axis('off')
    if(title!=''):
        plt.title(title)
    plt.show()


In [11]:
def remove_directory(directory_path):
    try:
        shutil.rmtree(directory_path)
    except FileNotFoundError:
        pass
    except Exception as e:
        pass

In [13]:
def binarize(rgb_image):


    # Convert the image to grayscale
    gray_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY)

    # Threshold the grayscale image to create a binary mask
    _, binary_mask = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)
    return binary_mask


In [14]:
def find_circularity(input_image):
  # mask = cv2.imread('1000masks/100.jpg',cv2.IMREAD_GRAYSCALE)
  mask=cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
  # mask=cv2.GaussianBlur(mask,(3,3),0)
  _, binary_mask = cv2.threshold(mask, 10, 255, cv2.THRESH_BINARY)


  # Find contours in the mask
  contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Convert the mask to BGR for visualization
  color_mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
  net_cicularity=[]
  for contour in contours:
      # Calculate area and perimeter of the contour
      area = cv2.contourArea(contour)
      perimeter = cv2.arcLength(contour, True)
      if area==0 or perimeter==0:
        circularity=0
        continue
      # Calculate circularity using the formula
      circularity = (4 * np.pi * area) / (perimeter ** 2)

      # Draw the contour on the color_mask image
      cv2.drawContours(color_mask, [contour], -1, (0, 255, 0), 2)  # Green color, thickness=2

      # Display circularity value on the image
      cv2.putText(color_mask, f"{circularity:.2f}", tuple(contour[0][0]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
      # title+=str(f"{circularity:.2f} ")
      net_cicularity.append(circularity)
  if len(net_cicularity)==0:
      return 0
  avg_circularity=sum(net_cicularity) / len(net_cicularity)
  return  color_mask,avg_circularity

In [15]:
def findVariance(image):
    _, stds = cv2.meanStdDev(image)
    return stds.flatten()

In [16]:
#Simple function to calc color vectors and circularity:
def find_features_color_circularity_variance(image):
  colorVector,_=findcolors(image)
  _,circularity=find_circularity(image)
  variance=findVariance(image)

  for i,val in enumerate(colorVector):
        if np.isnan(val):
            colorVector[i]==0.0
  if np.isnan(circularity):
        circularity=0.0
  for i,val in enumerate(variance):
        if np.isnan(val):
            variance[i]==0.0
  return np.concatenate((colorVector, [circularity*colorVector[1]], variance))

In [17]:
def showClassDiffnew(inp_all_masks,inp_labels,inp_classes):
  all_masks_array = np.array(inp_all_masks)
  labels_array = np.array(inp_labels)

  random_indices = np.random.permutation(len(inp_all_masks))
  all_masks_array = all_masks_array[random_indices]
  labels_array = labels_array[random_indices]

  # Set up the figure with subplots
  fig, axs = plt.subplots(len(inp_classes), 10, figsize=(25, 10))

  # Iterate over each class
  for i, class_label in enumerate(inp_classes):
      # Select 10 images for the current class
      class_images = all_masks_array[labels_array == class_label][:10]
      # Display each image in the corresponding subplot
      for j in range(10):
          axs[i, j].imshow(cv2.imread(masks_directory+'/'+class_images[j]))
          axs[i, j].axis('off')

          # # Add title with the class label
          axs[i, j].set_title(f'Class {class_label}')

  # Adjust layout to prevent clipping of titles
  plt.tight_layout()
  plt.show()

In [18]:
def remove_white_and_bone(original,remove):
    white_lower = np.array([170, 170, 170], dtype=np.uint8)
    white_upper = np.array([255, 255, 255], dtype=np.uint8)
    white_mask = cv2.inRange(original, white_lower, white_upper)

    inv_white_mask = cv2.bitwise_not(white_mask)
    inv_remove = cv2.bitwise_not(remove)
    inv_white_mask_minus_bone = cv2.bitwise_and(inv_white_mask, inv_white_mask, mask=inv_remove)

    return inv_white_mask_minus_bone

In [ ]:
# for imagepath in os.readfile('./content/input'):
imagepath='cellu.jpg'
image=cv2.imread(imagepath)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
pltshow(image,size=10)

In [ ]:
import time
start=time.time()
masks=returnMasks(image)
len(masks)
end=time.time()
print(end-start)

: 

In [ ]:
pltshow(image,annotations=True,size=10)

: 

In [ ]:
for mask in masks:
        mask_image = (mask['segmentation'] * 255).astype(np.uint8)
        result_image = cv2.bitwise_and(image, image, mask=mask_image)
        pltshow(result_image)

: 

In [ ]:
totalColorvec=[]
for mask in masks:
    mask_image = (mask['segmentation'] * 255).astype(np.uint8)
    result_image = cv2.bitwise_and(image, image, mask=mask_image)
    colorvec,title=findcolors(result_image)
    totalColorvec.append(colorvec)
    # pltshow(result_image,title=title)

: 

In [ ]:
#Kmean
from sklearn.cluster import KMeans
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
labels = kmeans.fit_predict(totalColorvec)
print("Cluster Labels:", labels)

: 

In [ ]:
for i,mask in enumerate(masks):
    mask_image = (mask['segmentation'] * 255).astype(np.uint8)
    result_image = cv2.bitwise_and(image, image, mask=mask_image)
    pltshow(result_image,title=labels[i])

: 

 **try 2, k means of the color features of all the masks**

In [ ]:

input_directory_path='/content/drive/MyDrive/ImagesData'
output_directory_path='/content/1000masks'
!mkdir $output_directory_path
allmasks=[]
allmaskfeatures=[]
i=0
for imagefile in os.listdir(input_directory_path):
    if imagefile=='.ipynb_checkpoints':
        continue
    image_path=os.path.join(input_directory_path, imagefile)

    image=cv2.imread(image_path)

    masks=returnMasks(image)
    print(i)
    for mask in masks:
        mask_image = (mask['segmentation'] * 255).astype(np.uint8)
        result_image = cv2.bitwise_and(image, image, mask=mask_image)
        allmasks.append(result_image)
        colorvec=findcolors(result_image)
        allmaskfeatures.append(colorvec)
        i+=1

labels=kmeans(totalColorvec,3)


: 

**try 3, Kmeans, heirarichal agglomerative clustering, DBSCAN on direct masks**

In [ ]:

input_directory_path='/content/drive/MyDrive/ImagesData'
output_directory_path='/content/1000masks'
!mkdir $output_directory_path
allmasks=[]
i=0
for imagefile in os.listdir(input_directory_path):
    if imagefile=='.ipynb_checkpoints':
        continue
    image_path=os.path.join(input_directory_path, imagefile)

    image=cv2.imread(image_path)

    masks=returnMasks(image)
    print(i)
    for mask in masks:
        mask_image = (mask['segmentation'] * 255).astype(np.uint8)
        result_image = cv2.bitwise_and(image, image, mask=mask_image)
        cv2.imwrite(output_directory_path+'/'+str(i)+'.jpg',result_image)
        i+=1


: 

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances_argmin_min

def load_images_from_directory(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(('.jpg', '.png', '.jpeg')):  # Add more extensions if needed
            path = os.path.join(directory, filename)
            image = cv2.imread(path)  # Load images in grayscale
            images.append(image)
    return np.array(images)

# Replace 'your_directory_path' with the actual path to your image directory
image_directory = '/content/1000masks'
X = load_images_from_directory(image_directory)
X_flatten = X.reshape((len(X), -1, 3))  # Assuming 3 channels (RGB)

# Apply Hierarchical Clustering
n_clusters = 3
agg_clustering = AgglomerativeClustering(n_clusters=n_clusters)
agg_labels = agg_clustering.fit_predict(X_flatten.reshape((len(X_flatten), -1)))


: 

In [ ]:
# # fig, axs = plt.subplots(n_clusters, 10, figsize=(15, 6))
num0,num1,num2=0,0,0

for i in agg_labels:
  if i ==0:
    num0+=1
  elif i ==1:
    num1+=1
  elif i ==2:
    num2+=1
print(num0,num1,num2)
#overpowered 1 cluster

: 

In [ ]:

dbscan = DBSCAN(eps=3, min_samples=500)  # Adjust eps and min_samples based on your data
dbscan_labels = dbscan.fit_predict(X_flatten.reshape((len(X_flatten), -1)))
print(np.unique(dbscan_labels))
# Get indices of images in each cluster

: 

**Try 4, improving feature vector of masks**

In [ ]:
all_masks=[]
for mask_name in os.listdir('/content/1000masks'):
  mask_path='/content/1000masks/'+mask_name
  mask=cv2.imread(mask_path)
  all_masks.append(mask)

: 

In [ ]:
#1st feature vector= the 3 colors

: 

In [ ]:
import time
start_time = time.time()
features_3colors=[]
for mask in all_masks:
  features,title=findcolors(mask)
  features_3colors.append(features)
labels3clr=kmeans(features_3colors,3)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Total time: {elapsed_time:.4f} seconds")

: 

In [ ]:
showClassDiff(all_masks,labels3clr,[0,1,2])

: 

**2nd feature vector pink,purple,white, circularity*white**

In [ ]:
for mask in all_masks[:20]:
  mask,circularity=find_circularity(mask)
  pltshow(mask,title=str(f"{circularity:.2f} "))

: 

In [ ]:
start_time = time.time()

features_3colors_circularity=[]
for mask in all_masks:
  features_3colors_circularity.append(find_features_color_circularity(mask))

labelscolorCircularity=kmeans(features_3colors_circularity,4)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Total time: {elapsed_time:.4f} seconds")


: 

In [ ]:
showClassDiff(all_masks,labelscolorCircularity,[0,1,2,3])

In [ ]:
#performig DBSCAN on 4features
dbscan = DBSCAN(eps=20, min_samples=500)  # Adjust eps and min_samples based on your data
dbscan_labels = dbscan.fit_predict(features_3colors_circularity)
showClassDiff(all_masks,dbscan_labels,[-1,0])

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
agglomerative_clustering = AgglomerativeClustering(n_clusters=4)
heir_cluster_labels = agglomerative_clustering.fit_predict(features_3colors_circularity)

# Plot the dendrogram (optional)
linkage_matrix = linkage(features_3colors_circularity, method='ward')
dendrogram(linkage_matrix)
plt.title('Hierarchical Clustering Dendrogram')
plt.show()

In [ ]:
showClassDiff(all_masks,heir_cluster_labels,[0,1,2,3])

In [ ]:
svmModel=joblib.load("./svm_color_circu_stds_91.joblib")

In [ ]:
imageBG=cv2.imread("./Individual-Masks-PGI-clean/PGI_image_0/bg/1.jpg")
imageCell=cv2.imread("./Individual-Masks-PGI-clean/PGI_image_0/cell/0.jpg")
imageBone=cv2.imread("./Individual-Masks-PGI-clean/PGI_image_1/bone/36.jpg")
imageFat=cv2.imread("./Individual-Masks-PGI-clean/PGI_image_0/fat/7.jpg")

In [ ]:
featureTesting=find_features_color_circularity_variance(imageBG)
svmModel.predict([featureTesting])

# Dataset making part 1 (from images to classes)


 Once we have images that we would like to automatically Annotate, we can use SAM to segment large and distinct areas of the images, these "segmentation objects" are delivered as binary masks. Each segmentation object/binary mask is representation of the region that SAM seperated. These segmentation objects are unclassified, so for actual segmentation we need to provide some class to them. Therefore we initially classified masks manually then used that data to train an SVM model that classifies masks based upon their features.  Let's first look at how SAM is converting images to segmentation objects and also saving their "feature vector".

In [ ]:
input_directory_path='images126' #input direcory contating images
output_directory_path='./output126' #output directory that will contain classified segmentations
masks_directory='pgi126' # a folder to contain all masks generated by all images for classification together
!mkdir  $output_directory_path $masks_directory

In [ ]:
featureVec=[] #saving features of each mask sequentially
masksPerImage=[] # a list of number of masks per image
mask_names=[] # names of each mask saved sequentially 
n_mask=0 #total number of masks

for n,imagefile in enumerate(os.listdir(input_directory_path)):
    print(n,end='-')
    image_name='PGI_image_'+str(n) # Naming the images using our nomenclature i.e. PGI_imaage_0,PGI_imaage_1.....
    image_path=os.path.join(input_directory_path,imagefile)


    # Our annotations will first be saved in a folder containing all image's annotations. the structureof this folder will be:
    #PGI_image_0
     #-->og.tif
     #-->bone
     #   |-->0.jpg
     #   |-->3.jpg
     #
     #-->bg
     #   |-->2.jpg
     #
     #-->fat
     #   |-->1.jpg
     #
     #-->cell
     #   |-->4.jpg
     #   |-->5.jpg
    output_dir=output_directory_path+'/'+image_name # The path and name of a new directory that will save this image's annotations
    remove_directory(output_dir) # removing the directory and its contents if it already exists

    os.mkdir(output_dir)
    os.mkdir(output_dir+'/bone') 
    os.mkdir(output_dir+'/bg')
    os.mkdir(output_dir+'/fat')
    os.mkdir(output_dir+'/cell')

    image=cv2.imread(image_path)
    cv2.imwrite(output_dir+'/og.tif',image) #Saving the original image as well in the folder, normally tif or jpg

    masks=returnMasks(image) # generating masks using SAM, the output is a list of binary image with white region as the segmented area/foreground and black being the rest of the image/background
    masksPerImage.append(len(masks))

    for mask in masks: # reading individual binary image
        mask_image = (mask['segmentation'] * 255).astype(np.uint8) # mask['segmentation'] contatins the acutal mask, which needs to be converted from 0,1 to 0,255 and then to uint8
        result_image = cv2.bitwise_and(image, image, mask=mask_image) # converting binary mask to a cutout of the original image, result_image is like the original image but with the parts that were background in segmentation object being black in this image too
        featureVec.append( find_features_color_circularity_variance(result_image) ) # finding the feature vector of the result_image, more explanation about featurevector in its respective function defination
        cv2.imwrite(masks_directory+'/'+image_name+'-'+str(n_mask)+'.jpg',result_image) #saving the mask as PGI_image_0-0.jpg
        mask_names.append(image_name+'-'+str(n_mask)+'.jpg') # appending mask name 0.jpg to mask_names
        n_mask+=1

In [38]:
# Feature Vector might have a NAN value therefore cionverting it to 0.0
for num, sublist in enumerate(featureVec):
    for i, value in enumerate(sublist):
        if np.isnan(value):
            featureVec[num][i] = 0.0

In [39]:
#Loading and opredicting class of each mask based on its feature vector
svmModel=joblib.load("svm_color_circu_stds_96.joblib")
labels=svmModel.predict(featureVec)

In [ ]:
showClassDiffnew(mask_names,labels,[0,1,2,3])

In [42]:
unique_values, counts = np.unique(labels, return_counts=True)
for value, count in zip(unique_values, counts):
    print(f"{value}: {count}")

0: 192
1: 2
2: 38


In [43]:
actualClass={0:'cell',
1:'bone',
2:'fat',
3:'bg'}
for i,mask_name in enumerate(mask_names):
  folder_name,file_name=mask_name.split('-')
  image=cv2.imread(masks_directory+'/'+mask_name)
  cv2.imwrite( output_directory_path+'/'+folder_name+'/'+actualClass[labels[i]]+'/'+file_name, image)



Now all the masks of each image are saved in each of their respective folder, we need to take a look at them to confirm that all masks are in their correct class, as they'll be converted to PascalVOC 

# Dataset making part 2 (To Pascal VOC)

We use annotaion platform called CVAT to edit our annotations, This platform needs annotations in a standard format such as PascalVOC, COCO or Cityscapes. PascalVOC is a instance segmenataion annoatatation format, it is popular and easy to manipulate, as it saves annotations in form of images, rather than boundary pixel coordinate in XML files or JSON files used by COCO format.
The following is the directory structure of PascalVOC:

    Annotations
        PGI_image_0.xml
        PGI_image_1.xml
    ImageSets
        Main
            default.txt
        Segmentation
            default.txt
    JPEGImages (optional)
        PGI_image_0.jpg
        PGI_image_1.jpg        
    SegmentationClass
        PGI_image_0.jpg
        PGI_image_1.jpg 
    SegmentationObject
        PGI_image_0.png
        PGI_image_1.png
    lablmap.txt

Explanation of each part:
1. **Annotations:**     contains xml file that has basic information of each image i.e. filename, width, height
2. **ImageSets:**     contains 2 txt files in Main and Segmentation, these files have only the name of each image
3. **JPEGImages:**     a folder contating all the original images
4. **SegmentationClass:**    a folder that contains a png file of the SegmenationClass image, In this image each pixel has the color of the class it belongs. for eg. all pixels that are cells will be coloured in a single color, in same fashion bg, bone and fat will also have a different color. So only 4 colors will be there in this image
5. **SegmentationObject:**     a folder that contains a png file of the SegmentationObject image, In this image each object that was recogonized distinctly is given a different color, eg. fat1, fat2, bone1 and bone2 are different objects of the classes, all such objects are coloured differently. This image can contain 256 different colors (MAX LIMIT OF PascalVOC)
6. **lablmap.txt:**     This file contains the color of each class in SegmentaionClass For eg. cell:128,0,0::

In [44]:
from lxml import etree

In [2]:
#Making the Xml file like the following example:
#     <annotation>
#        <folder/>
#            <filename>PGI_image_0.jpg</filename>
#        <size>
#            <width>1200</width>
#            <height>1920</height>
#            <depth/>
#        </size>
#        <segmented>1</segmented>
#     </annotation>

def makeXML(img, width, height):
    annotation = etree.Element('annotation')

    fo = etree.Element('folder')
    fo.text =  ''

    annotation.append(fo)

    f = etree.Element('filename')
    f.text = str(img+'.jpg')

    annotation.append(f)

    size = etree.Element('size')
    w = etree.Element('width')
    w.text = str(width)
    h = etree.Element('height')
    h.text = str(height)
    d = etree.Element('depth')
    d.text = str('')


    size.append(w)
    size.append(h)
    size.append(d)
    annotation.append(size)

    segmented=etree.Element('segmented')
    segmented.text='1'
    annotation.append(segmented)


    return annotation

In [60]:
cleanOutput='./output126' # input the folder where we saved our annotations  
OutputDir='PASCALoutput126/' # Folder that will contain the PascalVOC annotation
imagesFolder='images126/' # Folder that will contains only images
!mkdir $imagesFolder
!mkdir $OutputDir
!mkdir $OutputDir'SegmentationObject'
!mkdir $OutputDir'SegmentationClass'
!mkdir $OutputDir'Annotations'
!mkdir $OutputDir'ImageSets'
!mkdir $OutputDir'ImageSets/Main'
!mkdir $OutputDir'ImageSets/Segmentation'

In [72]:
class_color={
    'bone':[0,0,128],
    'cell':[128,128,0],
    'fat':[128,0,0],
    'bg':[0,128,0]
}

In [69]:
default="" #default.txt file in 'ImageSets/Main' and 'ImageSets/Segmentation'

#EDIT THIS TO BE IN COORDINATION WITH class_color
labelmap="# label:color_rgb:parts:actions\nbackground:0,0,0::\ncell:128,128,0::\nfat:128,0,0::\nbg:0,128,0::\nbone:0,0,128::" #labelmap.txt file


In [63]:
# converting the saved image back to binary mask
def binarize_for_pascal(rgb_image):
    gray_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray_image, 15, 255, cv2.THRESH_BINARY)
    return binary_mask


In [64]:
#Function to get the 256 colors that are allowed in PascalVOC SegmentationObjects
def color_map_get256(N=256):
    def bitget(byteval, idx):
        return ((byteval & (1 << idx)) != 0)

    cmap = np.zeros((N, 3), dtype=np.dtype)
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7-j)
            g = g | (bitget(c, 1) << 7-j)
            b = b | (bitget(c, 2) << 7-j)
            c = c >> 3

        cmap[i] = np.array([r, g, b])
    return cmap

color_map256=color_map_get256()

In [65]:
def addObject2Image(image,objects,color): # add an object to image with random color
    masked_image=image.copy()
    masked_image[objects==255]=color[::-1]
    return masked_image

In [66]:
def addClass2Image(image,mask,color_rgb):
    color_bgr=color_rgb[::-1]
    masked_image=image.copy()
    masked_image[mask==255]=color_bgr
    return masked_image

In [ ]:
default="" #making sure default file is empty before appending
outofLimitMaskClasses=[]
for n,dir in enumerate(os.listdir(cleanOutput)): #dir=MQ_image_0 or PGI_image_0
        print(dir)
        default+=dir+"\n" #adding image name to default.txt

        og_img=cv2.imread(cleanOutput+'/'+dir+'/og.tif')
        cv2.imwrite(imagesFolder+dir+'.tif',og_img) #adding image to image directory for data to be uploaded later

        #making annotation xml file for each image
        og_image_shape=og_img.shape
        annotation=makeXML(dir,og_image_shape[0],og_image_shape[1])
        save_path = os.path.join(OutputDir+'Annotations/'+dir+ ".xml")
        with open(save_path, 'wb') as file:
            aux = etree.tostring(annotation, pretty_print=True)
            aux.decode("utf-8")
            file.write(aux)

        image_objects=np.zeros(og_img.shape) #initiallly we use empty mask before overlaying individual masks on it
        image_classes=np.zeros(og_img.shape) #initiallly we use empty mask before overlaying combined class mask on it
        empty_mask=np.zeros_like(cv2.cvtColor(og_img,cv2.COLOR_BGR2GRAY)) #to save combined mask for each class

        combined_class={
            'bone': empty_mask,
            'bg': empty_mask,
            'fat': empty_mask,
            'cell': empty_mask
        }

        color_number_for_mask=2 #start colouring masks from color no. 2 in color_map256

        for item in os.listdir(cleanOutput+'/'+dir): # bone,cell,og.jpg
            item_path = os.path.join(cleanOutput+'/'+dir, item) # '/Individual-Masks/MQ_image_0/bone'

            if os.path.isdir(item_path) and  item!='cell': #open dirs other than cell and add their masks to combined_class and image_objects, SAM creates bad cell mask

                masks = os.listdir(item_path)
                if masks:
                    for mask_name in masks: # mask= 10.jpg, 11.jpg
                        # num_masks+=1
                        mask=cv2.imread(item_path+'/'+mask_name)
                        binary_mask=binarize_for_pascal(mask)

                        combined_class[item]=cv2.bitwise_or(combined_class[item],binary_mask)  #combine each class's mask
                        if len(masks)<200: # we want that each folder only have less than 200 masks then only we will allow it to add all its masks as different segmentation object, otherwise combine all the masks of that class together
                            if color_number_for_mask<255: #255 first masks + 1 cell mask
                                image_objects=addObject2Image(image_objects,binary_mask,color_map256[color_number_for_mask])
                                color_number_for_mask+=1
                            else:
                                print("Individiual masks for ",dir," more than 256, combining those masks") # CVAT and pascal VOC only allows 256 objects per image
                        else:
                            outofLimitMaskClasses.append(item)
        for maskClass in outofLimitMaskClasses:
            image_objects=addObject2Image(image_objects,combined_class[maskClass],class_color[maskClass])

        removeable_area=cv2.bitwise_or(combined_class['bone'], combined_class['bg'])
        removeable_area=cv2.bitwise_or(removeable_area, combined_class['fat'])
        combined_class['cell']= remove_white_and_bone(og_img,removeable_area) # cell mask is generated from scratch by taking the non white part - bone part
        image_objects=addObject2Image(image_objects,combined_class['cell'],color_map256[1])

        for mask_class in ['cell','bg','fat','bone']:
            image_classes=addClass2Image(image_classes,combined_class[mask_class],class_color[mask_class])
        cv2.imwrite(OutputDir+'SegmentationObject/'+dir+'.png',image_objects)
        cv2.imwrite(OutputDir+'SegmentationClass/'+dir+'.png',image_classes)

In [68]:
text_file = open(OutputDir+'ImageSets/Main/default.txt', "w")
text_file.write(default)
text_file.close()

text_file = open(OutputDir+'ImageSets/Segmentation/default.txt', "w")
text_file.write(default)
text_file.close()

text_file = open(OutputDir+'labelmap.txt', "w")
text_file.write(labelmap)
text_file.close()